## Function Calling

GPT 3.5 ve 4 modelleri, Haziran-Eylül 2021 ile Aralık 2023 ve öncesinden gelen verilerle eğitildiği için maalesef, bu modeller Haziran-Eylül 2021 veya Aralık 2023'den sonra ortaya çıkan ya da keşfedilen bilgileri bilemezler. Ayrıca, karmaşık matematik problemlerini çözmek için yetersizdirler. Bu durumda, modellerimiz, tanımadıkları veya zayıf oldukları konularda daha tutarlı ve doğru sonuçlar döndürebilmek için model için tanımladığımız fonksiyonu çağırarak kullanılabilir.

Örneğin, vücut kitle indeksi hesaplaması Nisan 2023'den sonra keşfedilmiş olsun veya modeller vücut kitle indeksini hesaplama konusunda çok kötü olsun. Modelin vücut kitle indeksini doğru bir şekilde hesaplaması için aşağıda bir fonksiyon oluşturalım ve sonra bu fonksiyonu GPT modellerinde nasıl çağırdığımıza bakalım.

Ayrıca bir text'den istediğimiz verileri çekmek içinde Function calling çok kullanışlıdır. Langchain dersinde örneğini göreceğiz.






Unfortunately, since the GPT 3.5 and 4 models are trained with data from June-September 2021 and December 2023 and before, they cannot know information that emerges or is discovered after June-September 2021 or December 2023. They are also inadequate for solving complex math problems. In this case, our models can be used by calling the function we define for the model to return more consistent and accurate results on topics they are unfamiliar or weak in.

For example, the calculation of body mass index was discovered after April 2023, or the models are very bad at calculating body mass index. Let's create a function below for the model to calculate BMI correctly and then see how we call this function in GPT models.

Function calling is also very useful for pulling the data we want from a text. We will see an example in the Langchain tutorial.

Translated with DeepL.com (free version)

In [1]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY']=userdata.get('openai_key')

In [3]:
from openai import OpenAI

client = OpenAI()

In [4]:
def body_mass_index(height,  weight):
  "calculate the body mass index"
  b_m_i=weight/(height*height)

  return f"The body mass index is about {b_m_i:.2f}"

# First, we define our function to calculate the body mass index.

In [5]:
body_mass_index_func = {
    "name": "body_mass_index",
    "description": "Calculates the body mass index.",
    "parameters": {
        "type": "object",
        "properties": {
            "height": {
                "type": "number",
                "description": "person's height in meters"
            },
            "weight": {
                "type": "number",
                "description": "person's weight in kilograms"
            }

        },
        "required": ["height",  "weight"]
    }
}

# Fonksiyonun adını, açıklamasını (kullanım amacını), fonksiyon parametrelerini ve türlerini(type) yukarıda gösterildiği gibi tanımlıyoruz. Burada
# en önemli kısım, yapılan açıklamalardır(description). Çünkü model, bir fonksiyonu çağırıp çağırmama kararını bu fonksiyon açıklamalarına dayalı olarak
# verir. Ayrıca, parametre açıklamalarına dayalı olarak model, metinden ilgili parametreleri seçebilir.

# Modelle sorduğunuz soru, fonksiyonun açıklamasına semantik(anlamsal) olarak çok yakınsa, model bu fonksiyonu çağıracaktır. Ancak, fonksiyon açıklaması
# kötü yapılmış veya eksikse, model fonksiyonu çağırmayacaktır.

# Model, fonksiyonu çağırdıktan sonra, doğru parametreleri metinden seçmek için iyi tanımlanmış parametre açıklamalarına sahip olmak önemlidir.
# Eğer parametre açıklamaları iyi yapılmamışsa veya eksikse, model textden doğru parametreleri seçemez.

# We define the function name, description (intended use), function parameters and type as shown above. Here we define the
# The most important part is the description. Because the model decides whether to call a function based on these function descriptions
# gives. Also, based on the parameter descriptions, the model can select relevant parameters from the text.

# If the question you ask the model is semantically very close to the function description, the model will call that function. However, the function description
# If it is poorly done or missing, the model will not call the function.

# After the model calls the function, it is important to have well-defined parameter descriptions to select the correct parameters from the text.
# If parameter descriptions are poorly done or missing, the model will not be able to select the correct parameters from the text.


In [7]:
type(body_mass_index_func)

dict

In [8]:
prompt = "What is the body mass index of a person who is 1800 mm tall and weight 80000 grams?"

res = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": prompt}],
    functions=[body_mass_index_func],
    function_call="auto" # "none"
)
res

# 'functions' parametresini 'body_mass_index_func' olarak ayarlayın.

# 'function_call' parametresini 'auto' olarak ayarlayın.

ChatCompletion(id='chatcmpl-9mzbHKFNJoMIj3ZGPuzDq8YVa7pS5', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"height":1.8,"weight":80}', name='body_mass_index'), tool_calls=None))], created=1721463563, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=87, total_tokens=108))

In [9]:
res.choices[0].message.function_call.arguments
# Prompt metninden seçilen parametreleri çekiyoruz.

# Gördüğünüz gibi, çıktımız bir JSON formatındadır (string).
# Python'da, JSON formatındaki veriyi doğrudan kullanamayız. Bu nedenle, json.loads() fonksiyonunu kullanarak veriyi dictionary formatına çevirerek
# Python'da JSON formatındaki veriyi kullanabiliriz.
# Veri dictionary formatına çevrildikten sonra, kolayca boy ve kilo bilgilerini çekebiliriz.

# We extract the selected parameters from the Prompt text.

# As you can see, our output is in JSON format (string).
# In Python, we cannot use JSON format data directly. Therefore, we need to convert the data to dictionary format using the json.loads() function
# In Python we can use data in JSON format.
# Once the data is converted to dictionary format, we can easily extract height and weight information.

'{"height":1.8,"weight":80}'

In [10]:
import json

name = res.choices[0].message.function_call.name
arguments = json.loads(res.choices[0].message.function_call.arguments) # Bir JSON dizesini bir dictionary çevirmek için json.loads() fonksiyonunu kullanın
                                                                       # Bu fonksiyon, JSON string'i bir dictionarye çevirir.
print(name)
print(arguments)

body_mass_index
{'height': 1.8, 'weight': 80}


In [11]:
print(arguments["height"])
print(arguments["weight"])

# boy ve kilo bilgilerini kolayca çekebildik

1.8
80


In [12]:
body_mass_index(arguments["height"],  arguments["weight"])

# tanımladığımız fonksiyon içine parametreleri yazınca istediğim outputu döndürüyor.

'The body mass index is about 24.69'

In [13]:
def run_conversation(prompt):

    response = client.chat.completions.create(model='gpt-3.5-turbo',
                                              messages=[{"role": "user", "content": prompt}],
                                              functions=[body_mass_index_func],
                                              function_call="auto")


    # Step 1, modelin bir fonksiyon çağırmak(function_calling) isteyip istemediğini kontrol edin
    if  response.choices[0].finish_reason == "function_call":
        # json string'i dictionary'e çevirin
        arguments = json.loads(response.choices[0].message.function_call.arguments)

        # Step 2, fonksiyonu çağrın ve bir değişkene atayın
        function_response = body_mass_index(arguments["height"],
                                            arguments["weight"])

        return function_response # fonksiyon sonucunu döndürün
    return response.choices[0].message.content # eğer fonksiyon çağrılmıyorsa. Chatgpt'nin kendi cevabını döndür.


# run_conversation fonksiyonuna verdiğimiz textden sonra model "finish_reason" olarak "function_call" döndürüyorsa,
# cevaplarını body_mass_index fonksiyonu üzerinden verecektir. Aksi halde model cevaplarını kendi yöntemiyle verecektir.

# If the model returns “function_call” as “finish_reason” after the text we give to the run_conversation function,
# will give its answers via the body_mass_index function. Otherwise the model will give its answers in its own way.

In [14]:
run_conversation("What is the body mass index of a person who is 1800 mm tall and weight 80000 grams?")

'The body mass index is about 24.69'

In [15]:
run_conversation("What is the Capital of Spain?")

'The capital of Spain is Madrid.'